In [ ]:
import numpy as np
import pandas as pd


df_return = pd.read_csv("Data/return_dataset.csv", index_col = 0)
df_return.head()

In [ ]:
# Change df_return column names to have 20xx-xx-xx form (year-month-day)
new_cols = list(pd.date_range(start = '2015-04-01', end = '2024-04-01'))
df_return.columns = new_cols

df_return.head()

In [ ]:
# Drop columns of all zero entries
all_zero_cols = (df_return==0).all()
all_zero_idx = all_zero_cols[all_zero_cols].index
df_return.drop(columns=all_zero_idx, inplace=True)
df_return.shape

In [ ]:
# Import 10 year US treasury bond yield data for risk free rate computation
df_bond = pd.read_csv("Data/10yr_UStreasury_yield.csv")
df_bond = df_bond.drop(columns=['Open', 'High', 'Low', 'Change %'])
df_bond = df_bond.T
df_bond.columns = df_bond.iloc[0] # Make date row into column index and remove date row
df_bond = df_bond[1:]
df_bond.columns = pd.to_datetime(df_bond.columns, format='%m/%d/%Y')

df_bond.head()


In [ ]:
# Combining df_return and df_bond 
# Compute common columns and make it list and sort
common_columns = set(df_return.columns) & set(df_bond.columns)
common_columns = sorted(list(common_columns))

# Sort df_bond according to the column index and select data from common_columns
df_bond.sort_index(axis=1, inplace=True)
df_bond = df_bond.loc[:,common_columns]

# Concatenating df_return and df_bond
df_result = pd.concat([df_return, df_bond])

# Rename some of row indices
change_dict = {'니케이225':'Nikkei225', 'Price':'10yr_yield'}
df_result.rename(index=change_dict, inplace=True)

df_result.head()

In [ ]:
# Make open and closed price data
KOSPI_200_list = ['005930', '000660', '373220', '207940', '005380', '000270', '068270', '005490', '105560', '035420', '006400', '051910', '028260', '055550', '003670', '035720', '012330', '032830', '086790', '138040', '066570', '000810', '015760', '259960', '003550', '329180', '018260', '033780', '011200', '034730', '009150', '323410', '034020', '017670', '024110', '012450', '096770', '316140', '402340', '010130', '042660', '090430', '267260', '030200', '009540', '010140', '352820', '047050', '003490', '010950', '051900', '450080', '326030', '086280', '005830', '022100', '001570', '000100', '251270', '011070', '161390', '241560', '034220', '047810', '010120', '267250', '097950', '028050', '009830', '377300', '006260', '302440', '011170', '029780', '006800', '011790', '032640', '021240', '180640', '004020', '361610', '078930', '064350', '128940', '005940', '036570', '011780', '000720', '001040', '071050', '079550', '271560', '272210', '016360', '035250', '039490', '003410', '010620', '018880', '000120', '004990', '002790', '001450', '138930', '383220', '001440', '000150', '175330', '036460', '052690', '088350', '002380', '028670', '081660', '004370', '012750', '008930', '008770', '282330', '003230', '030000', '073240', '112610', '020150', '014680', '007070', '000880', '103140', '023530', '000990', '010060', '026960', '204320', '009420', '139480', '004170', '007310', '111770', '042670', '192820', '298050', '051600', '047040', '017800', '004490', '298020', '011210', '005850', '145720', '000080', '000240', '139130', '009240', '006360', '002710', '185750', '336260', '375500', '280360', '006280', '137310', '069620', '014820', '004800', '069960', '005300', '161890', '005420', '009970', '004000', '003620', '000210', '006110', '003090', '001120', '001740', '120110', '285130', '192080', '039130', '001800', '006650', '300720', '271940', '105630', '114090', '020560', '178920', '010780', '008730', '069260', '001430', '009900', '093370', '001680', '000670', '032350', '005250', '019170', '016380']

open = pd.DataFrame()

for kospi in KOSPI_200_list:
    open_pd = pd.read_csv("KOSPI_200/{}.KS_final.csv".format(kospi))
    open_pd.drop(columns=['Close', 'Return'], inplace=True)
    open_pd.rename(columns={'Open' : kospi}, inplace=True)

    if not open.empty:
        open = open.merge(open_pd, on='Date', how='outer')
    else:
        open = open_pd

close = pd.DataFrame()

for kospi in KOSPI_200_list:
    closed_pd = pd.read_csv("KOSPI_200/{}.KS_final.csv".format(kospi))
    closed_pd.drop(columns=['Open', 'Return'], inplace=True)
    closed_pd.rename(columns={'Close' : kospi}, inplace=True)

    if not close.empty:
        close = close.merge(closed_pd, on='Date', how='outer')
    else:
        close = closed_pd

In [ ]:
# Check if there is a nan value
check_open = open.isnull().any().any()
check_close = close.isnull().any().any()
check_list = [check_open, check_close]

for check in check_list:
    if check:
        print("Nan value exists")
    else:
        print("No nan values")

In [ ]:
# Make open and close have the same format as df_result
open['Date'] = pd.to_datetime(open['Date'], format='%Y%m%d')

open = open.T
open.columns = open.iloc[0] 
open = open[1:]

close['Date'] = pd.to_datetime(close['Date'], format='%Y%m%d')

close = close.T
close.columns = close.iloc[0] 
close = close[1:]

In [ ]:
# drop the same columns as df_result
open.drop(columns=all_zero_idx, inplace=True)
close.drop(columns=all_zero_idx, inplace=True)

In [ ]:
# df_result.to_csv("whole_data.csv", index=False)
# open.to_csv("open_price.csv", index=False)
# close.to_csv("close_price.csv", index=False)